In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import glob
import os
from multiprocessing import Process
import sys
sys.path.append('..')
from modules import utils, constants
from sklearn.impute import KNNImputer

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Invalid MIT-MAGIC-COOKIE-1 key/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [2]:
SEED = constants.SEED
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.set_random_seed(SEED)
tf.compat.v1.set_random_seed(SEED)
SEED

42

In [3]:
def impute_missing_data(X_train):
    imputer = KNNImputer(n_neighbors=10)
    X_imputed = imputer.fit_transform(X_train)
    return X_imputed

In [4]:
constants.BETA, constants.CHECKPOINT_FREQ

(9, 1000000)

In [5]:
def run_dqn_model(model_type, steps):
    dir_name = f'seed_{SEED}_{steps}'
    parent_dir = f'../models/logs/{model_type}/missingness/0.1/no_step_penalty/knn_imputer'
    path = os.path.join(parent_dir, dir_name)
    os.mkdir(path)
  
    if model_type=='dqn':
        model = utils.stable_vanilla_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dqn', filename=f'dqn_{steps}')
    elif model_type=='ddqn':
        model = utils.stable_double_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='ddqn', filename=f'ddqn_{steps}')
    elif model_type== 'dueling_dqn':
        model = utils.stable_dueling_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_dqn', filename=f'dueling_dqn_{steps}')
    elif model_type == 'dueling_ddqn':
        model = utils.stable_dueling_ddqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_ddqn', filename=f'dueling_ddqn_{steps}')
    elif model_type =='dqn_per':
        model = utils.stable_vanilla_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dqn_per', filename=f'dqn_per_{steps}', per=True)
    elif model_type == 'ddqn_per':
        model = utils.stable_double_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='ddqn_per', filename=f'ddqn_per_{steps}', per=True)
    elif model_type == 'dueling_dqn_per':
        model = utils.stable_dueling_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_dqn_per', filename=f'dueling_dqn_per_{steps}', per=True)
    elif model_type == 'dueling_ddqn_per':
        model = utils.stable_dueling_ddqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_ddqn_per', filename=f'dueling_ddqn_per_{steps}', per=True)
    else:
        raise ValueError(f'Unknown model type - {model_type}!')
    return model

In [6]:
train_df = pd.read_csv('../new_data/train_set_missingness_0.1.csv')
# train_df = train_df.fillna(-1)
train_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,proteinuria,biopsy_proven_lupus_nephritis,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,1.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1
2,0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,...,1.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,3.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,1


In [7]:
train_df.label.value_counts()

0    25240
1    25160
Name: label, dtype: int64

In [8]:
train_df.iloc[90]

ana                              0.0
fever                            0.0
leukopenia                       0.0
thrombocytopenia                 0.0
auto_immune_hemolysis            0.0
delirium                         0.0
psychosis                        0.0
seizure                          0.0
non_scarring_alopecia            0.0
oral_ulcers                      0.0
cutaneous_lupus                  0.0
pleural_effusion                 0.0
pericardial_effusion             0.0
acute_pericarditis               0.0
joint_involvement                0.0
proteinuria                      0.0
biopsy_proven_lupus_nephritis    0.0
anti_cardioliphin_antibodies     0.0
anti_β2gp1_antibodies            0.0
lupus_anti_coagulant             0.0
low_c3                           1.0
low_c4                           0.0
anti_dsdna_antibody              0.0
anti_smith_antibody              NaN
label                            0.0
Name: 90, dtype: float64

In [9]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train = impute_missing_data(X_train)
X_train, y_train = np.array(X_train), np.array(y_train)
X_train.shape, y_train.shape

((50400, 24), (50400,))

In [10]:
# model_names = ['dqn', 'ddqn', 'dueling_dqn', 'dueling_ddqn', 'dqn_per', 'ddqn_per', 'dueling_dqn_per', 
#                'dueling_ddqn_per']
model_names = ['dueling_dqn_per', 'dueling_ddqn_per']
procs = []
steps = int(100e6)

In [11]:
# run_dqn_model(model_names[0],steps)

In [12]:
for name in model_names:
#     run_dqn_model(name, steps)
    proc = Process(target=run_dqn_model, args=(name, steps))
    procs.append(proc)
    proc.start()

The environment seed is [42]
The environment seed is [42]








Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where












Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where










--------------------------------------
| % time spent exploring  | 95       |
| episodes                | 100000   |
| mean 100 episode reward | -0.7     |
| steps                   | 482493   |
| success rate            | 0.16     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 95       |
| episodes                | 100000   |
| mean 100 episode reward | -0.6     |
| steps                   | 482398   |
| success rate            | 0.21     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 90       |
| episodes                | 200000   |
| mean 100 episode reward | -0.6     |
| steps                   | 980196   |
| success rate            | 0.23     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 90       |
| episodes                | 200000   |
| mean 100 episode reward | -0.6     |
| steps              

--------------------------------------
| % time spent exploring  | 21       |
| episodes                | 1600000  |
| mean 100 episode reward | -1       |
| steps                   | 8223131  |
| success rate            | 0.01     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 19       |
| episodes                | 1700000  |
| mean 100 episode reward | -1       |
| steps                   | 8442577  |
| success rate            | 0.02     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 16       |
| episodes                | 1300000  |
| mean 100 episode reward | -1       |
| steps                   | 8804506  |
| success rate            | 0.11     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 17       |
| episodes                | 1800000  |
| mean 100 episode reward | -0.9     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4300000  |
| mean 100 episode reward | -0.8     |
| steps                   | 13595917 |
| success rate            | 0.09     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4400000  |
| mean 100 episode reward | -0.8     |
| steps                   | 13792044 |
| success rate            | 0.09     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4500000  |
| mean 100 episode reward | -1       |
| steps                   | 13989750 |
| success rate            | 0        |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4600000  |
| mean 100 episode reward | -1       |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 7100000  |
| mean 100 episode reward | -0.6     |
| steps                   | 19172007 |
| success rate            | 0.21     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 7200000  |
| mean 100 episode reward | -1       |
| steps                   | 19374470 |
| success rate            | 0        |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2000000  |
| mean 100 episode reward | -0.5     |
| steps                   | 19905883 |
| success rate            | 0.37     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 7300000  |
| mean 100 episode reward | -1       |
| steps                  

...

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 15300000 |
| mean 100 episode reward | -0.8     |
| steps                   | 35949954 |
| success rate            | 0.1      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 15400000 |
| mean 100 episode reward | -0.8     |
| steps                   | 36157689 |
| success rate            | 0.1      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 15500000 |
| mean 100 episode reward | -0.9     |
| steps                   | 36366096 |
| success rate            | 0.04     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 15600000 |
| mean 100 episode reward | -0.7     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 18000000 |
| mean 100 episode reward | -0.6     |
| steps                   | 41640165 |
| success rate            | 0.19     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 18100000 |
| mean 100 episode reward | -0.9     |
| steps                   | 41853158 |
| success rate            | 0.06     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 18200000 |
| mean 100 episode reward | -1       |
| steps                   | 42068872 |
| success rate            | 0.01     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 18300000 |
| mean 100 episode reward | -1       |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 20800000 |
| mean 100 episode reward | -1       |
| steps                   | 47782138 |
| success rate            | 0        |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3800000  |
| mean 100 episode reward | -0.1     |
| steps                   | 49273581 |
| success rate            | 0.52     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 20900000 |
| mean 100 episode reward | -1       |
| steps                   | 48009000 |
| success rate            | 0        |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 21000000 |
| mean 100 episode reward | -0.6     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4200000  |
| mean 100 episode reward | -0.2     |
| steps                   | 55717088 |
| success rate            | 0.52     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 23500000 |
| mean 100 episode reward | -1       |
| steps                   | 54463928 |
| success rate            | 0.03     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 23600000 |
| mean 100 episode reward | -1       |
| steps                   | 54758190 |
| success rate            | 0        |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 23700000 |
| mean 100 episode reward | -1       |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 25300000 |
| mean 100 episode reward | -0.2     |
| steps                   | 74159309 |
| success rate            | 0.5      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 5500000  |
| mean 100 episode reward | 0        |
| steps                   | 76687605 |
| success rate            | 0.61     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 25400000 |
| mean 100 episode reward | -0.3     |
| steps                   | 75704111 |
| success rate            | 0.46     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 5600000  |
| mean 100 episode reward | -0       |
| steps                  

In [13]:
for proc in procs:
    proc.join()
print('All jobs completed and terminated successfully')

All jobs completed and terminated successfully


In [ ]:
# mylist = [{'name':30, 'list_feature': np.array([1,2,3]), 'label':0}, {'name':50, 'list_feature': np.array
#                                                                       ([4,5,6]), 'label':1}]
# a = pd.DataFrame(mylist)
# a

In [ ]:
# a.iloc[0]['list_feature']

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# X = a[['name', 'list_feature']]
# y = a['label']
# rf = RandomForestClassifier().fit(X, y)